# Tkinter App

In [1]:
from tkinter import *
from tkinter import ttk
from datetime import date
from tkinter import messagebox
import pandas as pd
import webbrowser
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup 
import os
import time
from functools import partial
from plyer import notification





class MonApp:
    def __init__(self,menu):
        self.menu=menu
        
        # datasets contenant les annonces
        self.df = pd.DataFrame()
        self.df2 = pd.DataFrame()
        self.df_ct = pd.DataFrame()
        
        
        # Création des boutons, des entrées des listes et des titres de la page principal
        self.lbl1=Label(menu,text="Nouvelle recherche",font=("Arial",30))
        self.lbl1.place(x=470,y=40)
        self.btn1=Button(menu,text="OK",bg='white',font=("Arial",13),width = 15,command=self.requete)
        self.btn1.place(x=560,y=550)
        
        self.lbl2=Label(menu,text="Prix Min:",font=('Arial',13,'bold'))
        self.lbl2.place(x=100,y=170)
        self.e=Entry(menu)
        self.e.place(x=100,y=200)
        
        self.lbl3=Label(menu,text="Prix Max:",font=('Arial',13,'bold'))
        self.lbl3.place(x=250,y=170)
        self.e1=Entry(menu)
        self.e1.place(x=250,y=200)
        
        self.lbl4=Label(menu,text="Kilométrage Min:",font=('Arial',13,'bold'))
        self.lbl4.place(x=100,y=300)
        self.e2=Entry(menu)
        self.e2.place(x=100,y=330)
        
        self.lbl5=Label(menu,text="Kilométrage Max:",font=('Arial',13,'bold'))
        self.lbl5.place(x=250,y=300)
        self.e3=Entry(menu)
        self.e3.place(x=250,y=330)
        
        
        self.lbl6=Label(menu,text="Année Min:",font=('Arial',13,'bold'))
        self.lbl6.place(x=100,y=400)
        self.e4=Entry(menu)
        self.e4.place(x=100,y=430)
        
        self.lbl7=Label(menu,text="Année Max:",font=('Arial',13,'bold'))
        self.lbl7.place(x=250,y=400)
        self.e5=Entry(menu)
        self.e5.place(x=250,y=430)
        
        list_carburant = ["Diesel","Essence", "Hybride", "Electrique", "Gaz", "Hydrogene"]
        self.lbl8=Label(menu,text="Carburant:",font=('Arial',13,'bold'))
        self.lbl8.place(x=500,y=170)
        self.listeCombo = ttk.Combobox(menu, values=list_carburant)
        self.listeCombo.place(x=500,y=200)
        
        list_type = ["Berlines","Breaks" ,"4x4-suv" ,"Citadines", "Monospaces","Coupé","Cabriolets"]
        self.lbl9=Label(menu,text="Type:",font=('Arial',13,'bold'))
        self.lbl9.place(x=500,y=400)
        self.listeCombo2 = ttk.Combobox(menu, values=list_type)
        self.listeCombo2.place(x=500,y=430)
        
        self.after_id = None
        
        self.url_argus = None
        self.url_ct = None
        
        self.previous_window = None

        
        
    
    def recherche_periodique(self):
        self.notification_pending = False
        # Appeler la fonction scraping avec les paramètres nécessaires
        self.scraping(self.url_argus, self.url_ct)
        
        
        # Planifier l'exécution de la fonction recherche_periodique après une 30 sec
        self.after_id = self.menu.after(30000, self.recherche_periodique) 
        
        
    
    
    def imprimer_ligne_selectionnee(self):
        # Imprime les valeurs de la ligne sélectionnée
        item = self.tree.selection()
        if item:
            link_value = self.tree.item(item, "values")[-2]
            print(link_value)
            # Ouvre le lien dans un navigateur web externe
            webbrowser.open(link_value)


    def scraping(self,lien1,lien2):
        headers ={"accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",'Accept-Encoding': 'gzip, deflate, br',"Accept-language":"fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7","Cache-control":"max-age=0","Sec-Fetch-Dest":"document","Sec-Fetch-User":"?1","user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Mobile Safari/537.36"}

        # scraping du site Argus
        response = requests.get(lien1,headers=headers)
        htmlpage2 = bs4.BeautifulSoup(response.text, "html")
        url=[]
        Model=[]
        Prix=[]
        Carburant=[]
        Km=[]
        Année=[]
        Critair=[]

        for item in htmlpage2.find('div', {'class' : 'list-group list'}).findAll({'a'})[::2]:
            url.append("https://occasion.largus.fr"+item.get("href")) #ajout de l'url à la liste
            Model.append(item.get("data-title").split("-")[0]) #ajout du model
            Prix.append(item.get("data-title").split("-")[1]) #ajout du prix

        for item in htmlpage2.find('div', {'class' : 'list-group list'}).findAll('div',{'class':'col-12 col-sm-9 bottom'}):
            Carburant.append(item.findAll("li")[0].text) #ajout du carburant
            Km.append(item.findAll("li")[1].text) #ajout du nombre de kilomètre
            Année.append(item.findAll("li")[2].text) #ajout de l'année
            if(item.findAll("li")[0].text == "Diesel"):
                if(int(item.findAll("li")[2].text)>=2011):
                    Critair.append(2)
                elif((int(item.findAll("li")[2].text)>=2006) & (int(item.findAll("li")[2].text)<=2010)):
                    Critair.append(3)
                elif((int(item.findAll("li")[2].text)>=2001) & (int(item.findAll("li")[2].text)<=2005)):
                    Critair.append(4)
                elif((int(item.findAll("li")[2].text)>=1997) & (int(item.findAll("li")[2].text)<=2000)):
                    Critair.append(5)
                else:
                    Critair.append(None)
                
            elif(item.findAll("li")[0].text == "Essence"):
                if(int(item.findAll("li")[2].text)>=2011):
                    Critair.append(1)
                elif((int(item.findAll("li")[2].text)>=2006) & (int(item.findAll("li")[2].text)<=2010)):
                    Critair.append(2)
                elif((int(item.findAll("li")[2].text)>=1997) & (int(item.findAll("li")[2].text)<=2000)):
                    Critair.append(3)
                else:
                    Critair.append(None)
            
            elif(item.findAll("li")[0].text == "Electrique"):
                Critair.append(0)
            else:
                Critair.append(1)
        # création du dataset avec les données de l'Argus
        self.df = pd.DataFrame({'Model': Model, 'Prix': Prix, 'Carburant': Carburant, 'Km': Km, "Année": Année,"url": url,"CritAir": Critair})
        
        # scraping de la centrale
        response_ct = requests.get(lien2,headers=headers)
        htmlpage_ct = bs4.BeautifulSoup(response_ct.text, "html")
        
        urlct=[]
        Modelct=[]
        Prixct=[]
        Caractéristique=[]
        Critairct=[]
        
        for item  in htmlpage_ct.find('div', {'class' : 'resultList'}).findAll({'a'}):
            urlct.append("https://www.lacentrale.fr"+item.get("href"))
            for i in item.find('div', {'class' : 'Vehiculecard_Vehiculecard_cardBody'}).findAll({'h2'}):
        #print(i.contents[0])
                Modelct.append(i.contents[0])
            for ii in item.find('div', {'class' : 'Vehiculecard_Vehiculecard_characteristics'}).findAll({'div'}):
        #print(ii.text)
                Caractéristique.append(ii.text)
        #print(caractéristique)
            for iii in item.find('div', {'class' : 'Text_Text_text Vehiculecard_Vehiculecard_priceContainer Text_Text_body3'}).find({'span'}):
        #print(iii)
                Prixct.append(iii+" €")
        
        Prixct=Prixct[::3]
        Annéect = Caractéristique[::4]
        Kmct = Caractéristique[1::4]
        Carburantct = Caractéristique[3::4]
        for i in range(len(Carburantct)):
            if(Carburantct[i] == "Diesel"):
                if(int(Annéect[i])>=2011):
                    Critairct.append(2)
                elif((int(Annéect[i])>=2006) & (int(Annéect[i])<=2010)):
                    Critairct.append(3)
                elif((int(Annéect[i])>=2001) & (int(Annéect[i])<=2005)):
                    Critairct.append(4)
                elif((int(Annéect[i])>=1997) & (int(Annéect[i])<=2000)):
                    Critairct.append(5)
                else:
                    Critairct.append(None)
                
            elif(Carburantct[i] == "Essence"):
                if(int(Annéect[i])>=2011):
                    Critairct.append(1)
                elif((int(Annéect[i])>=2006) & (int(Annéect[i])<=2010)):
                    Critairct.append(2)
                elif((int(Annéect[i])>=1997) & (int(Annéect[i])<=2000)):
                    Critairct.append(3)
                else:
                    Critairct.append(None)
            
            elif(Carburantct[i] == "Électrique"):
                Critairct.append(0)
            else:
                Critairct.append(1)
            
        # création du dataset avec les données de la centrale   
        self.df_ct = pd.DataFrame({'Model': Modelct, 'Prix': Prixct, 'Carburant': Carburantct, 'Km': Kmct, "Année": Annéect,"url": urlct, "CritAir": Critairct})
        
        #création du dataset contenant toutes les données et du fichier auto.csv 
        try:
            self.df2 = pd.read_csv('auto.csv')
            self.df2 = pd.concat([self.df_ct,self.df, self.df2], ignore_index=True)
            self.df2.to_csv('auto.csv', index=False)
        
        except:
            #self.df2 = self.df
            self.df2 = pd.concat([self.df_ct,self.df], ignore_index=True)
            self.df2.to_csv('auto.csv', index=False)
            
        
        self.new_window()
        
    # fenetre dans laquelle s'affiche les annonces après le scraping
    def new_window(self):
        
        if self.previous_window:
            self.previous_window.destroy()
        self.newWindow=Toplevel(self.menu)
        self.notification_pending = True

        # Afficher une notification
        notification_title = "AMP"
        notification_message = "Les annonces sont à jour."
        notification.notify(
            title=notification_title,
            message=notification_message,
            app_icon="cartransport_coch_7076.ico",
            timeout=3  # Temps en secondes avant que la notification se ferme automatiquement
        )
        self.previous_window = self.newWindow
        self.newWindow.title("Annonces")
        self.newWindow.geometry("1270x640+0+0")
        self.newWindow.iconbitmap("cartransport_coch_7076.ico")
        self.bouton=Button(self.newWindow,text="Ouvrir l'annonce",bg='white',font=("Arial",13),width = 15,command=self.imprimer_ligne_selectionnee)
        self.bouton.place(x=560,y=550)
        self.tree = ttk.Treeview(self.newWindow)
        self.tree["columns"] = list(self.df2.columns)
        for col in self.df2.columns:
            self.tree.heading(col, text=col)
            self.tree.column(col, anchor="w", width=170)
        for index, row in self.df2.iterrows():
            #self.tree.insert("", index, values=list(row))
            self.tree.insert("", index, values=list(row))
        self.tree.grid(row=0, column=0, sticky="nsew")
        
        scrollbar = ttk.Scrollbar(self.newWindow, orient=VERTICAL, command=self.tree.yview)
        self.tree.configure(yscroll=scrollbar.set)
        scrollbar.grid(row=0, column=1, sticky='ns')
        self.newWindow.protocol("WM_DELETE_WINDOW", self.fermer_application)
        #self.newWindow.after(30000, self.scraping(self.url_argus,self.url_ct))
        

    
    # suppression du fichier lors de la fermeture de l'app
    def fermer_application(self):
        
        fichier_csv = "auto.csv"

        # Vérifiez d'abord si le fichier existe avant de tenter de le supprimer
        try:
            os.remove(fichier_csv)
            
        except:
            return
        
        if self.after_id:
            self.menu.after_cancel(self.after_id)
        
        
        self.newWindow.destroy()
    
    
    
    # création des url des 2 sites pour la requete
    def requete(self):
        

        cpt = 0
        self.url_argus = "https://occasion.largus.fr/auto/"
        self.url_ct = "https://www.lacentrale.fr/listing"
        
        ###type###
        if(self.listeCombo2.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct +="?"
            if((self.listeCombo2.get()=="Berlines")|(self.listeCombo2.get()=="Breaks")):
                if(cpt == 0):
                    self.url_argus+="category=berlines-breaks"
                    self.url_ct+="categories=41_42"
                    cpt+=1
                else:
                    self.url_argus+="&category=berlines-breaks"
                    self.url_ct+="&categories=41_42"
            elif((self.listeCombo2.get()=="Coupé")|(self.listeCombo2.get()=="Cabriolets")):
                if(cpt == 0):
                    self.url_argus+="category=coupes-cabriolets"
                    self.url_ct += "categories=45"
                    cpt+=1
                else:
                    self.url_argus+="&category=coupes-cabriolets"
                    self.url_ct += "&categories=45"
            elif(self.listeCombo2.get()=="4x4-suv"):
                if(cpt == 0):
                    self.url_argus+="category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "categories=47"
                    cpt+=1
                else:
                    self.url_argus+="&category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "&categories=47"
            elif(self.listeCombo2.get()=="Citadines"):
                if(cpt == 0):
                    self.url_argus+="category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "categories=40"
                    cpt+=1
                else:
                    self.url_argus+="&category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "&categories=40"
            else:
            
                if(cpt == 0):
                    self.url_argus+="category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "&categories=44"
                    cpt+=1
                else:
                    self.url_argus+="&category={}".format(self.listeCombo2.get().lower())
                    self.url_ct += "&categories=44"
        print(self.url_argus)
        
        ###carburant###
        if(self.listeCombo.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct +="?"
            if((self.listeCombo.get()=="Diesel")|(self.listeCombo.get()=="Electrique")):
                if(cpt == 0):
                    self.url_argus+="energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="energies={}".format(self.listeCombo.get().lower()[:4])
                    cpt+=1
                else:
                    self.url_argus+="&energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="&energies={}".format(self.listeCombo.get().lower()[:4])
            elif(self.listeCombo.get()=="Essence"):
                if(cpt == 0):
                    self.url_argus+="energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="energies=ess"
                    cpt+=1
                else:
                    self.url_argus+="&energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="&energies=ess"
            elif(self.listeCombo.get()=="Hybride"):
                if(cpt == 0):
                    self.url_argus+="energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="energies=hyb%2Cplug_hyb%2Cnot_plug_hyb"
                    cpt+=1
                else:
                    self.url_argus+="&energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="&energies=hyb%2Cplug_hyb%2Cnot_plug_hyb"
            else:
                if(cpt == 0):
                    self.url_argus+="energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="energies=alt"
                    cpt+=1
                else:
                    self.url_argus+="&energy={}".format(self.listeCombo.get().lower())
                    self.url_ct+="&energies=alt"
                
        
        ###prix###
        if(self.e.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct +="?"
            try:
                prix_min = int(self.e.get())
                if(cpt == 0):
                    self.url_argus+="price%5Bmin%5D={}".format(prix_min)
                    self.url_ct+="priceMin={}".format(prix_min)
                    cpt+=1
                else:
                    self.url_argus+="&price%5Bmin%5D={}".format(prix_min)
                    self.url_ct+="&priceMin={}".format(prix_min)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer un Prix Minimum valide.")
                return
        
        if(self.e1.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct+="?"
            try:
                prix_max = int(self.e1.get())
                if(cpt == 0):
                    self.url_argus+="price%5Bmax%5D={}".format(prix_max)
                    self.url_ct+="priceMax={}".format(prix_max)
                    cpt+=1
                else:
                    self.url_argus+="&price%5Bmax%5D={}".format(prix_max)
                    self.url_ct+="&priceMax={}".format(prix_max)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer un Prix Maximum valide.")
                return
        print(self.url_argus)
        
        ###kilométrage###
        if(self.e2.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct +="?"
            try:
                km_min = int(self.e2.get())
                if(cpt == 0):
                    self.url_argus+="km%5Bmin%5D={}".format(km_min)
                    self.url_ct+="mileageMin={}".format(km_min)
                    cpt+=1
                else:
                    self.url_argus+="&km%5Bmin%5D={}".format(km_min)
                    self.url_ct+="&mileageMin={}".format(km_min)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer un Kilométrage Minimum valide.")
                return
                
        if(self.e3.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct +="?"
            try:
                km_max = int(self.e3.get())
                if(cpt == 0):
                    self.url_argus+="km%5Bmax%5D={}".format(km_max)
                    self.url_ct+="mileageMax={}".format(km_min)
                    cpt+=1
                else:
                    self.url_argus+="&km%5Bmax%5D={}".format(km_max)
                    self.url_ct+="&mileageMax={}".format(km_min)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer un Kilométrage Maximum valide.")
                return
                
        ###  Année ###
                
        if(self.e4.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct+="?"
            try:
                année_min = int(self.e4.get())
                if(année_min not in range(1900,2023)):
                    messagebox.showerror("Erreur", "L'année doit être comprise entre 1900 et 2023.")
                    return
                if(cpt == 0):
                    self.url_argus+="year%5Bmin%5D={}".format(année_min)
                    self.url_ct+="yearMin={}".format(année_min)
                    cpt+=1
                else:
                    self.url_argus+="&year%5Bmin%5D={}".format(année_min)
                    self.url_ct+="&yearMin={}".format(année_min)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer une Année Minimum valide.")
                return
                
        if(self.e5.get()!=""):
            if(self.url_argus == "https://occasion.largus.fr/auto/"):
                self.url_argus +="?"
                self.url_ct+="?"
            try:
                année_max = int(self.e5.get())
                if(année_max not in range(1900,2023)):
                    messagebox.showerror("Erreur", "L'année doit être comprise entre 1900 et 2023.")
                    return
                if(cpt == 0):
                    self.url_argus+="year%5Bmax%5D={}".format(année_max)
                    self.url_ct+="yearMax={}".format(année_max)
                    cpt+=1
                else:
                    self.url_argus+="&year%5Bmax%5D={}".format(année_max)
                    self.url_ct+="&yearMax={}".format(année_max)
            except: 
                messagebox.showerror("Erreur", "Veuillez entrer une Année Maximum valide.")
                return
                
        

        
        self.recherche_periodique()

        

        
            
        
        
        

        
        
        
        
    

        


        
            
window=Tk()
window.title("AMP")
window.iconbitmap("cartransport_coch_7076.ico")
window.geometry("1270x640+0+0")







mywin=MonApp(window)
window.mainloop()

https://occasion.largus.fr/auto/
https://occasion.largus.fr/auto/
